# Editionsdaten semantisch anreichern – XML-Parsing und Named Entity Recognition (NER) mit Python

Dieses Notebook ist im Rahmen der Fortbildungsreihe "Vom Dokument zur Edition" und konkret des Workshops [Editionsdaten semantisch anreichern – XML-Parsing und Named Entity Recognition (NER) mit Python](https://www.it.fu-berlin.de/unsere-services/kompetenzentwicklung/fortbildungen/workshops/E-Research/2026-01-27-Editionsdaten-WS4.html) im WiSe 2025/26 an der FU Berlin entstanden. Das Notebook soll als erster Einstieg dienen und erklären, wie XML-Dateien mit Python eingelesen sowie der enthaltene Text extrahiert, weiterverarbeitet und ins ursprüngliche XML zurückgeführt werden kann.

## Teil 1: XML einlesen und aufbereiten

### 1. Installation
Um dieses Notebook ausführen zu können, müssen [Python](https://www.python.org/) und [Jupyter Notebook](https://jupyter.org/) installiert sein. In Vorbereitung auf die Veranstaltung wird den Teilnehmenden eine darauf zugeschittene Anleitung zur Installation von Anaconda zugeschickt. Mit Anaconda sind neben Python bereits Jupyter Notebook sowie diverse weitere Bibliotheken vorinstalliert. Da die Bibliothek spacy (welche wir benötigen) leider nicht in der Liste der vorinstallierten Bibliotheken enthalten ist, erstellen wir eine seperate Entwicklungsumgebung. 

Hierzu öffnen wir ein Kommandozeilenfenster (*CMD.exe Prompt*) und geben die folgenden Befehle jeweils nacheinander ein:

* `conda create -n ner4xml python=3.11`
* `y`
* `conda activate ner4xml`
* `conda install jupyter spacy pandas`
* `jupyter notebook`

Kurzzusammenfassung, was hier passiert ist: wir (1) erstellen mit dem Paketmanager `conda` eine neue Umgebung namens `ner4xml`, welche die Python-Version 3.11 nutzt, mit (2) `y` (`yes`) bestätigen wir den Vorgang und sobald dies beendet ist (3) aktivieren wir die Umgebung, (4) installieren alle benötigten Bibliotheken und (5) rufen Jupyter Notebook auf. Dann müssen wir noch zu unserem Notebook navigieren und können dieses öffnen. Weitere Befehle, die sonst im Terminal eingegeben werden müssten (z.B. zur Installation weiterer Bibliotheken), können wir nun auch im Notebook aufrufen, indem wir ein `!` voranstellen.

In [1]:
#ein typischer Befehl, um zu prüfen ob und welche Version von Python installiert ist
!python --version

Python 3.9.23


Nun laden wir für später noch ein Modell von spacy herunter (diesen Schritt müssen wir nur einmal ausführen, dann können wir ihn z.B. mithilfe von `#` auskommentieren):

In [4]:
#!python -m spacy download de_core_news_sm

### 2. Import der Bibliotheken
Während die Bibliotheken grundsätzlich nur einmal installiert werden müssen, müssen wir sie jedes Mal vor der Verwendung importieren. Haben wir den Import vergessen, kommt z.B. eine Fehlermeldung wie `NameError: name '<name_bibliothek>' is not defined` 😡. Um das zu vermeiden, müssen wir alle Python-Bibliotheken importieren - auch solche aus der Standardbibliothek (wie hier `xml.etree.ElementTree`), die wir gar nicht gesondert vorher installieren mussten. Das Importieren erledigen wir am besten immer direkt am Anfang eines Python-Skripts oder -Notebooks ✅

In [5]:
import xml.etree.ElementTree as ET
import pandas as pd
import spacy

### 3. Laden und Parsen der XML-Daten
Nun geht es ans Eingemachte 🫙. Wir definieren eine Variable `path_to_file` und geben dort den relativen Pfad zur XML-Datei (kann abweichen und muss ggf. angepasst werden!) an. Mit `ET.parse()` wird die Datei geparsed (eingelesen) und mit `getroot()` sprechen wir den Wurzelknoten des XMLs an.

In [6]:
path_to_file = "data/input.xml"

tree = ET.parse(path_to_file)
root = tree.getroot()

Mit der `print()`-Funktion können wir eine Ausgabe erzeugen und uns hier beispielsweise das Element an der Wurzel anzeigen lassen. Wir sehen, dass vor dem Elementnamen noch eine Information über den Namespace enthalten ist: `{http://www.tei-c.org/ns/1.0}`. Das verwundert vielleicht zunächst, da es im XML-Dokument nicht an dieser Position zu lesen war. Damit ET 👽 weiß, dass wir uns auf diesen Namensraum beziehen, ohne dass wir nun jedes Mal diese Information voranstellen müssen, um einen Knoten ansprechen zu können, definieren wir eine Variable `namespaces` auf die wir später noch zugreifen werden. 

In [7]:
print(root)
print(root.tag)

namespaces={"":"http://www.tei-c.org/ns/1.0"}

<Element '{http://www.tei-c.org/ns/1.0}TEI' at 0x7921c7a99220>
{http://www.tei-c.org/ns/1.0}TEI


Nun sammeln wir alle Informationen, die wir gleich noch benötigen könnten, und legen dafür drei Listen an: `xml_nodes` für die Knoten der Elemente, auf die wir uns später beziehen und die wir ggf. auch aktualisieren wollen (alle, die in irgendeiner Weise Text enthalten), `xml_tags` für die Tag-Namen der gleichen Knoten und `xml_text` für den darin enthaltenen Text.

In [8]:
xml_nodes = []
xml_tags = []
xml_text = []

Jetzt starten wir unseren ersten Loop 💫 - glücklichweise bietet ET dafür auch direkt die Funktion `iter()` an. Wir iterieren durch alle (Kind-)Elemente des XML-Baums und speichern den mit `strip()` am Anfang und Ende von Leerzeichen befreiten Text dieser Elemente als `descendant_text`. Dann checken wir nochmal mit einer `if`-Bedingung ob auch wirklich Text enthalten ist und wenn dem so ist, fügen wir jeder der Listen je einen entsprechenden Eintrag hinzu. Am Ende checken wir nochmal ein paar Einträge und lassen uns die Gesamtlänge der ersten Liste (die anderen beiden müssten genau gleich lang sein) ausgeben.

In [9]:
for descendant in tree.iter():
      descendant_text = str(descendant.text).strip() #should probably be moved to if conditional
      if descendant_text != "" and descendant_text != "None":
            xml_nodes.append(descendant)
            xml_tags.append(descendant.tag)
            xml_text.append(descendant.text)

print([(node, tag, text) for (node, tag, text) in zip(xml_nodes[:3], xml_tags[:3], xml_text[:3])])
print(len(xml_nodes))

[(<Element '{http://www.tei-c.org/ns/1.0}title' at 0x7921c7a99720>, '{http://www.tei-c.org/ns/1.0}title', 'H. P. Lovecraft: Astronomical Observations (NAME)'), (<Element '{http://www.tei-c.org/ns/1.0}publisher' at 0x7921c7a99860>, '{http://www.tei-c.org/ns/1.0}publisher', 'tranScriptorium'), (<Element '{http://www.tei-c.org/ns/1.0}publisher' at 0x7921c7a999f0>, '{http://www.tei-c.org/ns/1.0}publisher', 'TRP document creator: fabian.etling@cedis.fu-berlin.de')]
412


### 4. Daten mit pandas aufbereiten

Der Vorteil unserer Listen ist, dass wir sie jetzt mithilfe der `pandas` Bibliothek in einen Dataframe, also eine tabellarische Struktur, überführen können. Das machen wir mit `pd.DataFrame`, hier können wir z.B. auch Namen für die Spalten (`columns`) mit angeben. Mit der `head()`-Funktion erhalten wir eine Übersicht über die ersten Einträge unseres Dataframes.

In [10]:
#import lists as cols to a pandas dataframe
xml_text_dataframe = pd.DataFrame(list(zip(xml_nodes, xml_tags, xml_text)), columns =["nodes", "tags", "text"])
xml_text_dataframe.head()

,nodes,tags,text
0,[],{http://www.tei-c.org/ns/1.0}title,H. P. Lovecraft: Astronomical Observations (NAME)
1,[],{http://www.tei-c.org/ns/1.0}publisher,tranScriptorium
2,[],{http://www.tei-c.org/ns/1.0}publisher,TRP document creator: fabian.etling@cedis.fu-b...
3,[],{http://www.tei-c.org/ns/1.0}desc,HPL uses a plus-like and sign to represent the...
4,[],{http://www.tei-c.org/ns/1.0}mapping,&


Seltsam - sind unsere `nodes` hier etwa verschwunden und nur noch leere Listen `[]` enthalten? Werfen wir nochmal einen genaueren Blick darauf und lassen uns den ersten Eintrag aus der Tabelle ausgeben:

In [11]:
print(xml_text_dataframe["nodes"][0])

<Element '{http://www.tei-c.org/ns/1.0}title' at 0x7921c7a99720>


Puh, alles noch da! 😅 When in doubt, use print: https://www.reddit.com/r/ProgrammerHumor/comments/ntro76/all_the_print_statements/

## Teil 2: Named Entity Recognition (NER) für XML 

### 5. NER auf einem Text anwenden
Um Named Entity Recognition (NER), also ein Verfahren zur Erkennung benannter Entitäten wie z.B. Personen-, Orts- oder Organisationsnamen, auf den textuellen Daten anzuwenden, müssen wir zunächst ein entsprechendes Modell laden. Wir haben bereits weiter oben das vortrainerte `de_core_news_sm` Modell von [SpaCy](https://spacy.io) installiert, jetzt laden wir dieses mit `spacy.load()`. Als Textbeispiel (`xml_text_example`) nehmen wir den Titel aus der ersten Zeile des DataFrames und wenden das Modell darauf an. Nun können wir uns beispielsweise für alle im Titel erkannten Entitäten deren Inhalt (`text`), Position (`start_char` = erstes Zeichen im Eingabetext, `end_char` = letztes Zeichen im Eingabetext) sowie das vergebene `label_` (z.B. `PER` für Person, `ORG` für Organisation) ausgeben lassen.

In [12]:
nlp = spacy.load("de_core_news_sm")
xml_text_example = xml_text_dataframe["text"][0]
print(xml_text_example)
doc = nlp(xml_text_example)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

H. P. Lovecraft: Astronomical Observations (NAME)
H. P. Lovecraft 0 15 PER
NAME 44 48 ORG


Es wurden zwei Entitäten im entsprechenden Text gefunden! Bei NAME handelt es sich wohl um einen Fehler, aber `H. P. Lovecraft` als Person auszuzeichnen erscheint sinnvoll. Um den zugehörigen Knoten im eingelesenen XML zu aktualisieren, definieren wir zwei weitere Variablen: `xml_title_node` und `per_to_be_added`. `xml_title_node` ist der Titelknoten aus der 1. Zeile des DataFrames und `per_to_be_added` die korrekt erkannte Person, die wir nun neu einbinden wollen.

In [13]:
xml_title_node = xml_text_dataframe["nodes"][0]
per_to_be_added = doc.ents[0]

Jetzt aktualisieren wir den Knoten und fügen ein NER-Tag hinzu. Dazu lesen wir zunächst den ursprünglichen Text bis zur Startposition der erkannten Entität aus. Dann nutzen wir `SubElement`, um ein Unterelement, in dem Fall `persName` hinzuzufügen. Anschließend erhält `persName` den Text der Entität und wir fügen nachfolgend den restlichen Text als `tail` hinzu.

In [14]:
xml_title_node.text = xml_text_example[0:per_to_be_added.start_char] #set text within node equal to all chars until PER entity starts
entity_tag = ET.SubElement(xml_title_node, 'persName') #add a nested tag for new entity
entity_tag.text = per_to_be_added.text #set entity tag text equal to ent.text for tag
entity_tag.tail = xml_text_example[per_to_be_added.end_char:len(xml_text_example)] #now, add the rest of the text        

Zum Schluss checken wir nochmal, ob persName innerhalb des `title`-Knotens entsprechend hinzugefügt wurde, indem wir ihn mit `root.findall()` auslesen und durch dessen Kindelemente iterieren: 

In [15]:
#check if node changed
for i in root.findall(".//title", namespaces)[0]:
    print(i)

<Element 'persName' at 0x7921fe5e2720>


### 6. NER auf viele Texte gleichzeitig anzuwenden

Wir können auch eigene Funktionen definieren, um unseren Code u.a. leichter wiederverwenden zu können und besser zu strukturieren. Hier ein einfaches Beispiel, wie so eine Funktion aussehen könnte. Für den Funktionsparameter `xml_table` können wir nun beispielsweise unseren `xml_text_dataframe` einsetzen und uns die ersten 5 Texteinträge aus der Tabelle ausgeben lassen:

In [16]:
def print_text(xml_table):
    for text in xml_table["text"]:
        print(text)

print_text(xml_text_dataframe[:5])

H. P. Lovecraft: Astronomical Observations (NAME)
tranScriptorium
TRP document creator: fabian.etling@cedis.fu-berlin.de
HPL uses a plus-like and sign to represent the conjunction "and".
&


In ähnlicher Weise können wir nun auch unseren bisherigen NER Code in einer Funktion unterbringen. Mit `enumerate()` werden die einzelnen `text` innerhalb des for-Loops durchgezählt und wir können die zugehörige Position (`index`) ansprechen - diese Position benötigten wir, um die entsprechenden NER-Tags an der jeweils richtigen Stelle im XML-Dokument einfügen zu können.

In [17]:
def ner_text(xml_table):
    for index, text in enumerate(xml_table["text"]):
        doc = nlp(text)
        for ent in doc.ents:
            #print(ent.text, ent.start_char, ent.end_char, ent.label_)
            current_xml_node = xml_table["nodes"][index]
            current_xml_node.text = text[0:ent.start_char] 
            entity_tag = ET.SubElement(current_xml_node, str(ent.label_) + '_name') 
            entity_tag.text = ent.text 
            entity_tag.tail = text[ent.end_char:len(text)]
        

ner_text(xml_text_dataframe)

### 7. Vergleich: NER auf plaintext anwenden

In [18]:
import re

with open("data/input.txt", encoding="utf-8") as file:
    textfile = file.read()

textfile = textfile.strip()
textfile = re.sub(r'\s+', ' ', textfile)

print(textfile)

Astronomical Observations Made By H.P. Lovecraft, 598, Angell St., Providence, R.I. U.S.A. Years: 1909 1910 1911 1912 1913 1914 1915 Comet, Halleys, Observed May 26, 1910 p. 1 Comet, Delavan’s, Observed September 16, 1914 p. 7 Eclipse of ☽︎ - June 3, 1909 p. a. Eclipse of ☽︎ - March 11-12, 1914 p. 6 Halley's Comet - May 26, 1910 p. 1 Halo around ☽︎, February 1, 1912 p 3. Mars, Occultation of, Observed September 1, 1909 p. 1 Moon, Eclipse of - June 3, 1909 p. a. Moon, Halo around, February 1, 1912 p. 3 Mercury near Moon - February 26, 1914 pp. 5-6 Occultation of Mars - September 1, 1909 p. 1 Paraselenae - Lunar Halo - February 1, 1912 p. 3 Special Observation June 3 - 1909 Moon’s Eclipse Clouds interfered, but several glimpses were obtained - Total 7.58 - ASTRONOMICAL OBSERVATIONS 1909 Begun September 1, 1909 September 1 - Observed Occultation of Mars by Moon’s bright limb at 8h 58m occultation took 40s. Ending at 9:57 was not observed. Shy hazy. Moon fair. Observed Mare Humorum Cleomed

In [19]:
doc_textfile = nlp(textfile)

for ent in doc_textfile.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Astronomical Observations Made By H.P. Lovecraft 0 48 PER
Angell St. 55 65 PER
Providence 67 77 ORG
R.I. U.S.A. 79 90 ORG
Comet 133 138 LOC
Halleys 140 147 LOC
Observed May 149 161 PER
Comet 176 181 MISC
Delavan’s 183 192 ORG
Observed 194 202 LOC
Eclipse 227 234 MISC
Eclipse of ☽ 262 274 MISC
Comet 310 315 MISC
May 318 321 LOC
☽ 348 349 MISC
Mars 374 378 LOC
Occultation 380 391 LOC
Observed 396 404 LOC
Eclipse 434 441 MISC
Lunar Halo - February 612 633 MISC
Occultation of Mars by Moon’s bright limb at 843 887 MISC
Ending at 917 926 MISC
Moon fair 960 969 MISC
Mars at 9h 1033 1043 MISC
Markings faint & ill 1045 1065 ORG
Thursday 1180 1188 LOC
May 1190 1193 LOC
Halley’s Comet 1206 1220 MISC
slightly less 1273 1286 ORG
Continued & 1337 1348 ORG
bright for about 15 1369 1388 MISC
traceable about 1402 1417 MISC
Nucleus & Coma 1511 1525 MISC
Used 1540 1544 ORG
Observed 1609 1617 LOC
Looking 1698 1705 LOC
θ ζ 1731 1734 MISC
Hydra 1911 1757 1767 MISC
Observed a lunar halo at about 1803 1833 OR

### 8. Ergebnisse exportieren
Am Ende kann der aktualisierte XML-Baum mit `tree.write()` wieder in eine XML-Datei geschrieben werden. Damit der Namensraum-Präfix nicht in den Elementen auftaucht, können wir ihn mit `register_namespace` vorher entsprechend angeben.

In [20]:
#write result to output file
ET.register_namespace("", "http://www.tei-c.org/ns/1.0")
tree.write('data/output.xml', xml_declaration=True, encoding='utf-8', method='xml')
#TODO: add PI/statement from L2: <?xml-model href="hpl_ao.rng" type="application/xml" schematypens="http://relaxng.org/ns/structure/1.0"?>

**Challenges/Next steps:**
- [ ] adapt other NER tools, including tokenization etc.
- [ ] check schemas, include statement from line 2 etc. (`lxml`?)
- [x] deal with multiple, nested annotations -> one function
- [ ] whitespaces
- [x] larger text sections, at least sentences
- [ ] decide how to deal with pre-existing annotations (removing, updating, adding new annotations?)